In [1]:
from pathlib import Path
import numpy as np
from sim import rebforces
from amuse.units import units

gas_profile_root = Path("disk/calculated_profiles/")
gas_profile_name = "20220721"
gas_profile = np.load(gas_profile_root / gas_profile_name / "all_variables.npz")

# rebforces.set_profiles(
#     len(gas_profile["r"]),
#     gas_profile["r"],
#     gas_profile["velocity"].T * rebforces.CM_PER_S,
#     gas_profile["rho_0"] * rebforces.G_PER_CM3,
#     1.0
# )

# rebforces.copy_np_to_c(gas_profile["sigma"] * rebforces.G_PER_CM3 / 14959787070000, rebforces.SURFACE_DENSITY_PROF, rebforces.STD_PROF_N.value)
# rebforces.copy_np_to_c(gas_profile["H"] / gas_profile["r_cm"], rebforces.SCALE_HEIGHT_PROF, rebforces.STD_PROF_N.value)
# rebforces.copy_np_to_c(gas_profile["torque"], rebforces.TORQUE_PROF, rebforces.STD_PROF_N.value)

In [2]:
import rebound
from collections import namedtuple

EARTH_MASS = (1 | units.MEarth).value_in(units.MSun)

TestOutput = namedtuple("TestOutput", ["t", "dt", "a", "rDa", "e", "rDe", "i", "rDi", "Dh"])

class TestSimulation(rebound.Simulation):
    def __init__(self, m=EARTH_MASS, rho_p=np.float64(3.) * rebforces.G_PER_CM3, a=0.25, e=0.0, inc=0.0, output_dt=None):
        super().__init__(self)
        
        self.G = 4 * np.pi ** 2
        self.integrator = "IAS15"

        sun = rebound.Particle(m=1.)
        self.add(sun)

        p = rebound.Particle(simulation=self, primary=self.particles[0], m=m, r=(3 * m / (4 * np.pi * rho_p))**(1./3), a=a, e=e, inc=inc)
        self.add(p)

        if output_dt is None:
            self.output_dt = self.particles[1].P
        else:
            self.output_dt = output_dt

        self.old_t = self.t
        self.old_orbit = self.particles[1].calculate_orbit()

    def __iter__(self):
        while True:
            self.integrate(self.t + self.output_dt)

            dt = self.t - self.old_t
            new_orbit = self.particles[1].calculate_orbit()

            def rD(attr):
                val = getattr(new_orbit, attr)
                if val == 0.0:
                    return 0
                else:
                    return (val - getattr(self.old_orbit, attr)) / dt / val

            ret = TestOutput(
                t = self.t,
                dt = dt,
                a = new_orbit.a,
                rDa = rD("a"),
                e = new_orbit.e,
                rDe = rD("e"),
                i = new_orbit.inc,
                rDi = rD("inc"),
                Dh = (new_orbit.h - self.old_orbit.h) / dt
            )

            yield ret

            self.old_t = self.t
            self.old_orbit = new_orbit

In [3]:
import numpy as np
from sim import rebforces
from amuse.units import units

def set_simple_profile(a, alpha, Sigma, eta, torque, C_D, h):
    r = np.linspace(a-0.05, a+0.05, rebforces.__STD_PROF_NMAX__)
    velocity = np.stack([(2 * np.pi / np.sqrt(r)) * (1 - 2 * eta)**2, np.zeros_like(r)])
    Sigma = Sigma * np.exp(-alpha * (r - a))
    h = np.full_like(r, h)
    rho_0 = Sigma / (h * r * np.sqrt(2*np.pi))
    torque = np.full_like(r, torque)

    rebforces.set_profiles(
        rebforces.__STD_PROF_NMAX__,
        r,
        velocity,
        rho_0 * rebforces.G_PER_CM3,
        C_D
    )

# rebforces.set_profiles(
#     len(gas_profile["r"]),
#     gas_profile["r"],
#     gas_profile["velocity"].T * rebforces.CM_PER_S,
#     gas_profile["rho_0"] * rebforces.G_PER_CM3,
#     1.0
# )

    rebforces.copy_np_to_c(Sigma * rebforces.G_PER_CM3 / 14959787070000, rebforces.SURFACE_DENSITY_PROF, rebforces.STD_PROF_N.value)
    rebforces.copy_np_to_c(h, rebforces.SCALE_HEIGHT_PROF, rebforces.STD_PROF_N.value)
    rebforces.copy_np_to_c(torque, rebforces.TORQUE_PROF, rebforces.STD_PROF_N.value)

In [4]:
a = 0.26
i = np.argmin(np.abs(gas_profile["r"] - a))

a = gas_profile["r"][i]
Sigma = gas_profile["sigma"][i]
torque = 0.05
C_D = 0.44
eta = 0.002
h = gas_profile["H"][i] / gas_profile["r_cm"][i]

lnSigma = np.log(gas_profile["sigma"])
alpha = -np.gradient(lnSigma, gas_profile["r"])[i]

set_simple_profile(a, alpha, Sigma, eta, torque, C_D, h)

In [5]:
rebforces.__STD_PROF_NMAX__

2048

In [6]:
from itertools import islice

test_sim = TestSimulation(a=a)
for i in islice(test_sim, 10):
    print(i)


TestOutput(t=0.13379381437671553, dt=0.13379381437671553, a=0.26159199999999994, rDa=-1.5860604675492848e-15, e=9.49259427713451e-18, rDe=7.474187088981242, i=0.0, rDi=0, Dh=0.0)
TestOutput(t=0.26758762875343106, dt=0.13379381437671553, a=0.261592, rDa=1.5860604675492844e-15, e=1.0502106671261862e-16, rDe=6.798613817751354, i=0.0, rDi=0, Dh=0.0)
TestOutput(t=0.40138144313014656, dt=0.1337938143767155, a=0.261592, rDa=0.0, e=1.310415177380151e-16, rDe=1.4841229135662548, i=0.0, rDi=0, Dh=0.0)
TestOutput(t=0.5351752575068621, dt=0.13379381437671556, a=0.26159199999999994, rDa=-1.5860604675492844e-15, e=1.910042808380027e-16, rDe=2.346402435669381, i=0.0, rDi=0, Dh=0.0)
TestOutput(t=0.6689690718835777, dt=0.13379381437671556, a=0.261592, rDa=1.586060467549284e-15, e=3.4748464140203054e-16, rDe=3.365799092264432, i=0.0, rDi=0, Dh=0.0)
TestOutput(t=0.8027628862602932, dt=0.13379381437671556, a=0.26159199999999994, rDa=-1.5860604675492844e-15, e=3.769362927933728e-16, rDe=0.5839903367941671,

In [ ]:
from itertools import islice

test_sim = TestSimulation(a=a)

test_sim.additional_forces = rebforces.IOPF_drag_all
test_sim.force_is_velocity_dependent = 1

for i in islice(test_sim, 10):
    print(i)


: 

: 